Load jupyter sql extension


In [2]:
"""
%load_ext sql
%config SqlMagic.autocommit = False
"""

'\n%load_ext sql\n%config SqlMagic.autocommit = False\n'

Next load the database

In [3]:
"""
%sql sqlite:///../data/raw/frantamarit.db
"""

'\n%sql sqlite:///../data/raw/frantamarit.db\n'

In [4]:
"""
%%sql tables <<

SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';
"""

"\n%%sql tables <<\n\nSELECT \n    name\nFROM \n    sqlite_master \nWHERE \n    type ='table' AND \n    name NOT LIKE 'sqlite_%';\n"

In [5]:
"""
tables.DataFrame()
"""

'\ntables.DataFrame()\n'

In [6]:
"""
%%sql master_table <<

SELECT 
*
FROM rank_info
JOIN personal_info ON rank_info.id = rank_info.id
JOIN business_info ON business_info.id = rank_info.id;
"""

'\n%%sql master_table <<\n\nSELECT \n*\nFROM rank_info\nJOIN personal_info ON rank_info.id = rank_info.id\nJOIN business_info ON business_info.id = rank_info.id;\n'

1. Creating engine to connect to the database

In [7]:
from sqlalchemy import create_engine
import pandas as pd

In [8]:
"""
def get_dataframe(x):
    eng = create_engine(x)
    tables = eng.table_names()
    for i in enumerate(tables):
        i = pd.read_sql_table(i, con=eng)
    return 
    
get_dataframe('sqlite:///../data/raw/frantamarit.db')
"""

"\ndef get_dataframe(x):\n    eng = create_engine(x)\n    tables = eng.table_names()\n    for i in enumerate(tables):\n        i = pd.read_sql_table(i, con=eng)\n    return \n    \nget_dataframe('sqlite:///../data/raw/frantamarit.db')\n"

2. Create engine and get data tables

In [9]:
#Review path
eng = create_engine('sqlite:///../data/raw/frantamarit.db')
table_names = eng.table_names()
print(table_names)

['business_info', 'personal_info', 'rank_info']


3. Creating data frames for each table

In [10]:
rank_info = pd.read_sql_table('rank_info', con=eng)
personal_info = pd.read_sql_table('personal_info', con=eng)
business_info = pd.read_sql_table('business_info', con=eng)

In [11]:
print(rank_info.notnull().sum())
print(personal_info.notnull().sum())

id            2260
name          2260
position      2208
Unnamed: 0    2260
dtype: int64
id            2208
lastName      2208
age           2143
Unnamed: 0    2208
gender        2195
country       2208
image         2208
dtype: int64


In [28]:
rank_info.drop('Unnamed: 0', axis=1, inplace=True)
personal_info.drop('Unnamed: 0', axis=1, inplace=True)
business_info.drop('Unnamed: 0', axis=1, inplace=True)

In [31]:
personal_info.head()

,id,lastName,age,gender,country,image
0,6528,beZOS,54 years old,M,None,https://specials-images.forbesimg.com/imageser...
1,5875,gaTES,62 years old,None,United States,https://specials-images.forbesimg.com/imageser...
2,8760,buFFEtt,87 years old,None,United States,https://specials-images.forbesimg.com/imageser...
3,4685,aRNAULT,69 years old,M,None,https://specials-images.forbesimg.com/imageser...
4,8584,zUCKERBERG,1985,M,United States,https://specials-images.forbesimg.com/imageser...


In [37]:
partial_info = pd.merge(rank_info, personal_info, on='id', how='inner')
master_info = pd.merge(partial_info, business_info, on='id', how='inner')
master_info.head()


,id,name,position,lastName,age,gender,country,image,Source,worth,worthChange,realTimeWorth,realTimePosition
0,6528,jeff BEZos,1.0,beZOS,54 years old,M,None,https://specials-images.forbesimg.com/imageser...,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,NaN,1
1,5875,bILL GATes,2.0,gaTES,62 years old,None,United States,https://specials-images.forbesimg.com/imageser...,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,NaN,2
2,8760,warREN BUFFett,3.0,buFFEtt,87 years old,None,United States,https://specials-images.forbesimg.com/imageser...,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,-0.002 millions USD,NaN,3
3,4685,bERNARD ARnault,4.0,aRNAULT,69 years old,M,None,https://specials-images.forbesimg.com/imageser...,Fashion & Retail ==> LVMH,72.0 BUSD,0.0 millions USD,NaN,4
4,8584,mark ZUCKERberg,5.0,zUCKERBERG,1985,M,United States,https://specials-images.forbesimg.com/imageser...,Technology ==> Facebook,71.0 BUSD,0.0 millions USD,NaN,5


In [69]:
#Cleaning and lowercasing name and lastName features
master_info[['name','lastName']] = \
master_info[['name','lastName']].apply(lambda x: x.astype(str).str.lower())

KeyError: "None of [Index(['name', 'lastName'], dtype='object')] are in the [columns]"

In [63]:
master_info.isnull().sum()

id                     0
full_name              0
position               0
last_name              0
age                   65
gender                13
country                0
image                  0
source                 0
worth                  0
worthChange            0
realTimeWorth       2208
realTimePosition       0
dtype: int64

In [60]:
#Renaming feature names
master_info.rename(index={e: e+1 for e in range(len(master_info))}, 
                columns={'name': 'full_name', 
                         'lastName': 'last_name',
                        'Source': 'source'}, 
                inplace=True)

In [61]:
master_info.head()

,id,full_name,position,last_name,age,gender,country,image,source,worth,worthChange,realTimeWorth,realTimePosition
1,6528,jeff bezos,1.0,bezos,54 years old,M,None,https://specials-images.forbesimg.com/imageser...,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,NaN,1
2,5875,bill gates,2.0,gates,62 years old,None,United States,https://specials-images.forbesimg.com/imageser...,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,NaN,2
3,8760,warren buffett,3.0,buffett,87 years old,None,United States,https://specials-images.forbesimg.com/imageser...,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,-0.002 millions USD,NaN,3
4,4685,bernard arnault,4.0,arnault,69 years old,M,None,https://specials-images.forbesimg.com/imageser...,Fashion & Retail ==> LVMH,72.0 BUSD,0.0 millions USD,NaN,4
5,8584,mark zuckerberg,5.0,zuckerberg,1985,M,United States,https://specials-images.forbesimg.com/imageser...,Technology ==> Facebook,71.0 BUSD,0.0 millions USD,NaN,5
